In [38]:
from dotenv import load_dotenv

In [5]:
import os

In [3]:
load_dotenv()

True

In [6]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [41]:
import pyodbc

try:
    conn = pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};"
                          "SERVER=LAPTOP-RES6G37K;"
                          "DATABASE=adventureworks2019;"
                          "UID=YourLogin;"
                          "PWD=Rsri@Nov1983;")
    print("✅ SQL Server Login Successful!")
    conn.close()
except Exception as e:
    print("❌ SQL Server Login Failed:", e)

❌ SQL Server Login Failed: ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'YourLogin\'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "adventureworks2019" requested by the login. The login failed. (4060); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'YourLogin\'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "adventureworks2019" requested by the login. The login failed. (4060)')


In [8]:
from sqlalchemy import create_engine

In [10]:
from sqlalchemy.engine import URL
server = "LAPTOP-RES6G37K"
database = "atliq_tshirts"
username = "MyLogin"
password = "Rsri@Nov1983"

connection_url = URL.create(
    "mssql+pyodbc",
    username=username,
    password=password,
    host="tcp:" + server,
    port=1433,
    database=database,
    query={"driver": "ODBC Driver 17 for SQL Server"}
)

engine = create_engine(connection_url)


In [11]:
try:
    with engine.connect() as conn:
        print("✅ SQLAlchemy connected successfully!")
except Exception as e:
    print("❌ SQLAlchemy connection failed:", e)

✅ SQLAlchemy connected successfully!


In [12]:
from langchain_openai import ChatOpenAI

In [13]:
import os
from langchain_openai import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.utilities import SQLDatabase
from sqlalchemy import create_engine

In [14]:
# Create SQLDatabase instance
db = SQLDatabase(engine)

In [24]:
print(db.get_usable_table_names())

['discounts', 't_shirts']


In [25]:
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

# ✅ Create SQLDatabaseChain
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# Example: Convert Natural Language to SQL
nl_query = "how many total blue nike t-shirts are available?"

# ✅ Generate SQL Query & Execute
result = db_chain.invoke(nl_query)

print("Query Result:", result)



> Entering new SQLDatabaseChain chain...
how many total blue nike t-shirts are available?
SQLQuery:SELECT SUM([stock_quantity]) as Total_Blue_Nike
FROM [t_shirts]
WHERE [brand] = 'Nike' AND [color] = 'Blue'
SQLResult: [(None,)]
Answer:There are no blue Nike t-shirts available.
> Finished chain.
Query Result: {'query': 'how many total blue nike t-shirts are available?', 'result': 'There are no blue Nike t-shirts available.'}


In [27]:
print(result)

{'query': 'how many total blue nike t-shirts are available?', 'result': 'There are no blue Nike t-shirts available.'}


In [29]:
db_chain

SQLDatabaseChain(verbose=True, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input', 'table_info', 'top_k'], input_types={}, partial_variables={}, template='You are an MS SQL expert. Given an input question, first create a syntactically correct MS SQL query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the TOP clause as per MS SQL. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in square brackets ([]) to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n

In [30]:
db_chain.invoke(nl_query)



> Entering new SQLDatabaseChain chain...
how many total blue nike t-shirts are available?
SQLQuery:SELECT SUM([stock_quantity]) as Total_Blue_Nike
FROM [t_shirts]
WHERE [brand] = 'Nike' AND [color] = 'Blue'
SQLResult: [(None,)]
Answer:There are no blue Nike t-shirts available.
> Finished chain.


{'query': 'how many total blue nike t-shirts are available?',
 'result': 'There are no blue Nike t-shirts available.'}

In [32]:
# Example: Convert Natural Language to SQL
nl_query = "how many total price for adidas ?"
result=db_chain.invoke(nl_query)



> Entering new SQLDatabaseChain chain...
how many total price for adidas ?
SQLQuery:SELECT SUM([price]) as TotalPrice
FROM [t_shirts]
WHERE [brand] = 'Adidas'
SQLResult: [(2160,)]
Answer:The total price for Adidas is 2160.
> Finished chain.


In [40]:
def query_answer(username,password,query):
    db = SQLDatabase(create_db_connection(username,password))
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    # ✅ Create SQLDatabaseChain
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
    # Example: Convert Natural Language to SQL
    # ✅ Generate SQL Query & Execute
    result = db_chain.invoke(query)
    return result 

In [39]:
server = "LAPTOP-RES6G37K"
database = "adventureworks2019"


def create_db_connection(username,password):
    connection_url = URL.create(
        "mssql+pyodbc",
        username=username,
        password=password,
        host="tcp:" + server,
        port=1433,
        database=database,
        query={"driver": "ODBC Driver 17 for SQL Server"}
    )

    engine = create_engine(connection_url)
    return engine 

In [35]:
result=query_answer("MyLogin","Rsri@Nov1983","how many total price for adidas ?")



> Entering new SQLDatabaseChain chain...
how many total price for adidas ?
SQLQuery:SELECT SUM([price]) as TotalPrice
FROM [t_shirts]
WHERE [brand] = 'Adidas'
SQLResult: [(2160,)]
Answer:The total price for Adidas is 2160.
> Finished chain.


In [36]:
result

{'query': 'how many total price for adidas ?',
 'result': 'The total price for Adidas is 2160.'}